# EPFL - Statistical learning (MATH-412) - Week 5
## K-fold cross validation

In [1]:
import os
import numpy as np
import utils as ut
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
from sklearn.metrics import mean_squared_error as mse
import sklearn.model_selection as model_selection
import csv

First we load the 'bodyfat.csv' dataset

In [3]:
# Load the dataset
input_file = os.path.join(os.getcwd(), 'data',  'bodyfat.csv')
file = open(input_file, 'rt')
reader = csv.reader(file, delimiter=';')
bodyfat = np.array([row for row in reader])

# Extract the header
header = bodyfat[0,:]

# Remove rows 2 and 4
col_to_del = [1, 3]
bodyfat = np.delete(bodyfat[1:,:], col_to_del, axis=0)

# Extract targets and features
targets = bodyfat[:,0].astype(np.float64)
features = np.array(bodyfat[:,1:]).T.astype(np.float64)

Here is the list of regularization parameters we will explore to select the best value

In [4]:
list_alphas = np.logspace(-6, 6, 200)

We create the LASSO model from which we will optimize the regularization parameter

In [5]:
# Lasso model
lasso = lm.Lasso(max_iter=1000, tol=1e-4, random_state=5)

We apply the K-fold cross validation and the one standard error rule on the LASSO model we've just created

In [6]:
# K-fold cross validation for Lasso with different values of the regularization parameter
K = 10
err_k_lasso = ut.k_fold_cross_valid(K, list_alphas, lasso, features, targets)
cv_error_lasso = np.mean(err_k_lasso, axis=1)

# Find the best value of lambda for K-fold CV
best_alpha = list_alphas[np.argmin(cv_error_lasso)]
min_cv_error_lasso = np.min(cv_error_lasso)
print('LASSO - Best lambda - K-fold CV: {}'.format(best_alpha))
print('LASSO - Best error - K-fold CV: {}'.format(min_cv_error_lasso))

# Standard deviation of the CV error
avg_err_lasso = np.mean(err_k_lasso, axis=0)
std_cv_lasso = 1/np.sqrt(K)*np.sqrt(np.mean((err_k_lasso-avg_err_lasso)**2, axis=1))

# Best lambda - one standard error rule
list_alphas_ose = list_alphas[list_alphas > best_alpha]
cv_error_ose_lasso = cv_error_lasso[list_alphas > best_alpha]
list_alphas_ose = list_alphas_ose[cv_error_ose_lasso < min_cv_error_lasso + std_cv_lasso[np.argmin(cv_error_lasso)]]
cv_error_ose_lasso = cv_error_ose_lasso[cv_error_ose_lasso < min_cv_error_lasso + std_cv_lasso[np.argmin(cv_error_lasso)]]

print('LASSO - Standard error CV: {}'.format(std_cv_lasso[np.argmin(cv_error_lasso)]))
print('LASSO - Best lambda - K-fold CV - One standard-error rule: {}'.format(list_alphas_ose[-1]))
print('LASSO - One standard-error rule - error: {}'.format(cv_error_ose_lasso[-1]))



LASSO - Best lambda - K-fold CV: 0.07663410868007446
LASSO - Best error - K-fold CV: 19.51243866367376
LASSO - Standard error CV: 6.320731205540608
LASSO - Best lambda - K-fold CV - One standard-error rule: 13.049019780144016
LASSO - One standard-error rule - error: 25.43040070163812


We repeat the same study for the ridge regression model

In [7]:
# Create the ridge model
ridge = lm.Ridge()

# Apply the K-fold cross validation
err_k_ridge = ut.k_fold_cross_valid(K, list_alphas, ridge, features, targets)
cv_error_ridge = np.mean(err_k_ridge, axis=1)

# Find the best value of lambda for K-fold CV
best_alpha_ridge = list_alphas[np.argmin(cv_error_ridge)]
min_cv_error_ridge = np.min(cv_error_ridge)
print('Ridge - Best lambda - K-fold CV: {}'.format(best_alpha))
print('Ridge - Best error - K-fold CV: {}'.format(min_cv_error_ridge))

# Standard deviation of the CV error
avg_err_ridge = np.mean(err_k_ridge, axis=0)
std_cv_ridge = 1/np.sqrt(K)*np.sqrt(np.mean((err_k_ridge-avg_err_ridge)**2, axis=1))

# Best lambda - one standard error rule
list_alphas_ose_ridge = list_alphas[list_alphas > best_alpha]
cv_error_ose_ridge = cv_error_ridge[list_alphas > best_alpha]
list_alphas_ose_ridge = list_alphas_ose_ridge[cv_error_ose_ridge < min_cv_error_ridge + std_cv_ridge[np.argmin(cv_error_ridge)]]
cv_error_ose_ridge = cv_error_ose_ridge[cv_error_ose_ridge < min_cv_error_ridge + std_cv_ridge[np.argmin(cv_error_ridge)]]

print('Ridge - Standard error CV: {}'.format(std_cv_ridge[np.argmin(cv_error_ridge)]))
print('Ridge - Best lambda - K-fold CV - One standard-error rule: {}'.format(list_alphas_ose_ridge[-1]))
print('Ridge - One standard-error rule - error: {}'.format(cv_error_ose_ridge[-1]))


Ridge - Best lambda - K-fold CV: 0.07663410868007446
Ridge - Best error - K-fold CV: 19.565259119377238
Ridge - Standard error CV: 1.4150194115667525
Ridge - Best lambda - K-fold CV - One standard-error rule: 965.8832241158708
Ridge - One standard-error rule - error: 20.83452511351617


In [ ]:
# Display the results
plt.subplot(121)
plt.plot(list_alphas, cv_error_ridge)
plt.scatter(best_alpha_ridge, min_cv_error_ridge, label='CV - Min. error')
plt.scatter(list_alphas_ose_ridge[-1], cv_error_ose_ridge[-1], label='CV - one standard error rule')
ax = plt.gca()
ax = plt.gca()
ax.set_xscale('log')
ax.set_xlabel('Values of the regularization parameter')
ax.set_ylabel('Cross-validation error')
ax.legend()
plt.title('CV error for Ridge')

plt.subplot(122)
plt.plot(list_alphas, cv_error_lasso)
plt.scatter(best_alpha, min_cv_error_lasso, label='CV - Min. error')
plt.scatter(list_alphas_ose[-1], cv_error_ose_lasso[-1], label='CV - one standard error rule')
ax = plt.gca()
ax = plt.gca()
ax.set_xscale('log')
ax.set_xlabel('Values of the regularization parameter')
ax.set_ylabel('Cross-validation error')
plt.title('CV error for Lasso')
ax.legend()
plt.show()
